In [109]:
import pandas as pd
train = pd.read_csv('train.csv', sep=',', index_col=0,
                       dtype = {'row_id': 'int64', 'timestamp': 'int64', 'user_id': 'int32', 'content_id': 'int16', 
                                'content_type_id': 'int8', 'task_container_id': 'int16', 'user_answer': 'int8', 
                                'answered_correctly': 'int8', 'prior_question_elapsed_time': 'float32', 
                                'prior_question_had_explanation': 'boolean'
                               }
                      )

train.head()
for i in range(train.shape[0]):
    if pd.isna(train['prior_question_had_explanation'][i]) == True:
        train['prior_question_had_explanation'][i] = False
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].astype('int8')

train.info()
unique_list = []
for col in train.columns:
    item = (col, train[col].nunique(), train[col].dtype)
    unique_list.append(item)
unique_counts = pd.DataFrame(unique_list,
                             columns=['Column_Name', 'Num_Unique', 'Type']
                            ).sort_values(by='Num_Unique',  ignore_index=True)
display(unique_counts)


# 0-вопросы, 1-лекции
train['content_type_id'].value_counts(normalize=True) 
train['content_type_id'].value_counts()[0] 
train['content_type_id'].value_counts()[1] 

#Соотношение правильных и неправильных
train[train['answered_correctly'] != -1]['answered_correctly'].value_counts(normalize=True) 
train[train['answered_correctly'] != -1]['answered_correctly'].value_counts()[1] 
train[train['answered_correctly'] != -1]['answered_correctly'].value_counts()[0]
train[train['answered_correctly'] != -1]['answered_correctly'].mean()
# 66% составляют правильные ответы, 34% - неправильные
# всего 65244627 правильных ответов и 34026673 неправильных ответов
# 0.657 - средний балл студента

#Среднее время на решение
train['prior_question_elapsed_time'].mean()
# Cреднее число отвеченных вопросов: 252.17
# Среднее кол-во правильных ответов: 165.74
# Среднее число просмотренных лекций: 4.97
# Среднее число виденных объяснений: 227.82
# Среднее время, затрачиваемое на вопрос: 13005
pd.DataFrame({'timestamp': train[train['answered_correctly'] != -1]['timestamp'].groupby(train['answered_correctly']).mean(),
              'prior_questions_time': train[train['answered_correctly'] != -1]['prior_question_elapsed_time'].groupby(train['answered_correctly']).mean(),
              'had_explanation': train[train['answered_correctly'] != -1]['prior_question_had_explanation'].groupby(train['answered_correctly']).sum()
             }
            )

# Можно сделать следующие выводы:
# - чем ближе к началу сессии вопрос, тем менее успешно его проходят студенты 
# (т.е. успешность коррелирует со временем, проведенным на платформе)
# - студенты, тратящие меньше времени на решение, чаще отвечают правильно
# - если студент видел объяснение предыдущий задачи, вероятность правильного ответа удваивается


#Основные выводы по разделу 1:
#Всего в данных присутствует информация об активности 393656 студентов.
#На платформе 13523 вопросов (объединенных в 10000 блоков) и 418 лекций по 7 темам.
#В среднем на ответ на вопрос у студентов уходит
#98% активновти студентов приходится на ответы на вопросы, только 2% татится на просмотр лекций.
#66% составляют правильные ответы, 34% - неправильные.
#0.657 - средний балл студента
#Cреднее число отвеченных вопросов: 252.17
#Среднее кол-во правильных ответов: 165.74
#Среднее число просмотренных лекций: 4.97
#Среднее число виденных объяснений: 227.82
#Среднее время, затрачиваемое на вопрос: 13005
#Cреднее время на решение вопроса 13005 милисекунд (= 13 секунд)
#Чем ближе к началу сессии вопрос, тем менее успешно его проходят студенты (т.е. успешность коррелирует со временем, проведенным на платформе)
#Студенты в среднем тратят меньше времени на правильный ответ, чем на неправильный (возможно, меньше сомневаются)
#Если студент видел объяснение предыдущий задачи, он в 2 раза чаще отвечает правильно, чем неправильно

#Новый DF
sudents_list = list(train['user_id'].unique())
users_q = len(sudents_list)
#Сокращении до 500
sudents_list = sudents_list[:500]
#Время работы каждого студента
time = []
for student in sudents_list:
    t = train[train['user_id'] == student]['timestamp'].max()
    time.append(t)
#сколько всего ответил на вопросы
ques_quant = []
for student in sudents_list:
    q = train[(train['user_id'] == student) & (train['content_type_id'] == 0)]['content_type_id'].count()
    ques_quant.append(q)
    
#Средний балл студента
av_grade = []
for student in sudents_list:
    g = train[(train['user_id'] == student) & (train['answered_correctly'] != -1)]['answered_correctly'].mean()
    av_grade.append(g)
#Просмотренные лекции
lec_watched = []
for student in sudents_list:
    l = train[train['user_id'] == student]['content_type_id'].sum()
    lec_watched.append(l)
    
#Кол-ва объяснений на вопросы
expl_watched = []
for student in sudents_list:
    e = train[train['user_id'] == student]['prior_question_had_explanation'].sum()
    expl_watched.append(e)
    
#Среднее время на вопрос
ques_time = []
for student in sudents_list:
    qt = train[(train['user_id'] == student)]['prior_question_elapsed_time'].mean()
    ques_time.append(qt)
    
#Первые 500 студентов  
students = pd.DataFrame({'user_id': sudents_list,
                        'time': time,
                        'ques_quant': ques_quant,
                        'av_grade': av_grade,
                        'lec_watched': lec_watched,
                        'expl_watched': expl_watched,
                         'ques_time': ques_time
                       }
                      )
students = students.astype({'user_id': 'int32',
                            'time': 'int64',
                            'ques_quant': 'int16', 
                            'av_grade': 'float32',
                            'lec_watched': 'int16',
                            'expl_watched': 'int16',
                            'ques_time': 'float32'
                           })

students.head()
#Сравнение выборки с оригиналом
students.describe()

# Мы видим, что наша выборка средними значениями отличается от общего датасета:
# средняя успеваемость у нашей группы ниже, чем у полного набора данных, 
# при этом среднее число просмотренных лекций и объяснений выше
# (за счет относительно небольшого кол-ва очень активно учащихся студентов
# это видно из смещения кол-ва отвеченных вопросов, просмотренных лекций и объяснений в верхний квартиль
# и высокой дисперсии)
# Однако все значения укладываются в среднеквадратическое отклонение, 
# поэтому можем считать их несущественными для результатов анализа


#Гистограмма распределения средниц оценок

students['av_grade'].hist(bins=20)

students['av_grade'].median()


students[students['time'] < 3000000].plot.scatter(x='av_grade', y='time')

# Из графика неочевидно, что с ростом времени, затраченного на платформе, растет процент правильных ответов у студентов
# Можем сделать 2 предположения:
# - либо время, проведенное на платформе не имеет значимого влияния на успеваемость
# - либо для эффекта нужно затрачивать время сильно больше, чем это делает большинство студентов
# (возможно студенты слишком рано бросают занятия)


#Когда студент бросает занятия

students[students['time'] < 3000000].hist(column=['time'])

# Мы видим, что после 1000000 милисекунд (=17 минут) 1/5 часть студентов не продолжает обучения, 
# после 2500000 милисекунд (=42 минуты) учебу бросает уже треть
# Получается, то, что мы изначально сочли выбросами - это наша основная рабочая выборка, которую надо исследовать,
# чтобы понять влияние факторов на успеваемость. 

# строка для проверки данных отсечения
students[students['time'] < 2500000].shape
#Построим диаграмму рассеяния для студентов, не бросивших учебу в течение 1го часа.
students[students['time'] > 3000000].plot.scatter(x='av_grade', y='time')

# Здесь уже становится видна тенденция роста успеваемости с течением веремени
#разбиение студентов по временным группам:
# - Добавим столбец с временной группировкой
# - Построим диаграмму размаха
def time_convert(e):
    if e < students['time'].quantile(0.1): return 0
    elif e < students['time'].quantile(0.2): return 1
    elif e < students['time'].quantile(0.3): return 2
    elif e < students['time'].quantile(0.4): return 3
    elif e < students['time'].quantile(0.5): return 4
    elif e < students['time'].quantile(0.6): return 5
    elif e < students['time'].quantile(0.7): return 6
    elif e < students['time'].quantile(0.8): return 7
    elif e < students['time'].quantile(0.9): return 8
    else: return 9
    
students['time_group'] = students['time'].apply(time_convert)

students.boxplot(column=['av_grade'], by=['time_group'])

# Первый эффект от занятий становится виден, если студент не бросает заниматься
# При этом, если студент потратил на занятия не менее 55 часов, его оценка становится устойчиво выше средней
# и продолжает расти со временем.
# Однако, если курс не пройден за ~ полгода, оценка становится нестабильной
# Т.е. стоит не только стимулировать студентов не бросать занятия после первого подхода,
# но и закончить курс за первые полгода обучения.

students['time'].quantile(0.7)/1000/60/60/8

#зависимость правильных ответов от кол-ва сделанных заданий
#Ограничим выборку студентами, не бросившими обучение сразу.

students[(students['ques_quant'] < 1000) & (students['time'] > 3000000)].plot.scatter(x='av_grade', y='ques_quant', c='red')

# чтобы убрать выбросы, ограничим данные сверху 1000
# На графике довольно очевидна связь успеваемости с кол-вом отвеченных вопросов
# Стоит провести группировку вопросов по кол-ву для дальнейшего анализа

#группировка по количеству отвеченных вопросов
def ques_convert(e):
    if e < students['ques_quant'].quantile(0.1): return 0
    elif e < students['ques_quant'].quantile(0.2): return 1
    elif e < students['ques_quant'].quantile(0.3): return 2
    elif e < students['ques_quant'].quantile(0.4): return 3
    elif e < students['ques_quant'].quantile(0.5): return 4
    elif e < students['ques_quant'].quantile(0.6): return 5
    elif e < students['ques_quant'].quantile(0.7): return 6
    elif e < students['ques_quant'].quantile(0.8): return 7
    elif e < students['ques_quant'].quantile(0.9): return 8
    else: return 9
    
students['q_group'] = students['ques_quant'].apply(ques_convert)


students.boxplot(column=['av_grade'], by=['q_group'])

# Мы видим странную просадку в 4м квартиле
students['ques_quant'].quantile(0.4)
# Можем предположить, что первые вопросы являются сильным демотиватором для учащегося.
# Но если студент отвечает больше, чем на 34 вопроса, он постепенно начинает улучшать свои показатели
# И дальше результаты только улучшаются
# На гистограме видно, что 32 вопроса - порог отсечения для многих студентов из нашей выборки

students[(students['ques_quant'] < students['ques_quant'].quantile(0.5))].hist(column=['ques_quant'])
#зависимость правильных ответов от кол-ва виденных объяснений
students[(students['expl_watched'] < 500)].plot.scatter(x='av_grade', y='expl_watched', c='yellow')
#группировка по правильным ответам
def expl_convert(e):
    if e < students['expl_watched'].quantile(0.1): return 0
    elif e < students['expl_watched'].quantile(0.2): return 1
    elif e < students['expl_watched'].quantile(0.3): return 2
    elif e < students['expl_watched'].quantile(0.4): return 3
    elif e < students['expl_watched'].quantile(0.5): return 4
    elif e < students['expl_watched'].quantile(0.6): return 5
    elif e < students['expl_watched'].quantile(0.7): return 6
    elif e < students['expl_watched'].quantile(0.8): return 7
    elif e < students['expl_watched'].quantile(0.9): return 8
    else: return 9
    
students['e_group'] = students['expl_watched'].apply(expl_convert)

students['expl_watched'].quantile(0.5)


# здесь мы видим, что 34 вопроса - порог отсечения для многих студентов
students[(students['expl_watched'] < students['expl_watched'].quantile(0.8))].hist(column=['expl_watched'])


students.boxplot(column=['av_grade'], by=['e_group'])

# Видно четкий тренд на повышение оценки с ростом кол-ва просмотренных объяснений
#зависимость правильных ответов от лекций
students[students['lec_watched'] < 21].plot.scatter(x='av_grade', y='lec_watched', c='green')

# Мы видим, что экстремальные выбросы можно убрать, ограничив число 21 лекцией

students[students['lec_watched'] < 21].boxplot(column=['av_grade'], by=['lec_watched'])

# Мы видим, что даже просмотр 1 леции повышает средний балл и значительно повышает минимальную оценку студента.
# Максимальный эффект достигается при просмотре 4-11 лекций, после чего эффективность просмотров снижается



#Основные выводы по разделу 2
#Мы сделали выборку из общего объема данных по 500 студентам. С учетом проведенного анализа мы можем сделать следующие выводы:

#Треть студентов бросает учебу, проведя на платформе менее 42 минут. За это время, вероятно, они не успевают почувствовать эффект, либо им просто что-то не нравится (недостаточно мотивации / слишком сложные вопросы / неудобный интерфейс и т.д.).

#Если студент потратил на занятия не менее 55 часов, его оценка становится устойчиво выше средней и продолжает расти со временем.

#Если курс не пройден за ~ полгода, оценка становится нестабильной. Т.е. стоит не только стимулировать студентов не бросать занятия после первого подхода, но и закончить курс за первые полгода обучения.

#Анализ данных по кол-ву отвеченных вопросов подсказывает, что первые вопросы сильно демотивируют учащихся. Большая часть не преодолевает порог в 32 вопроса. Следует стимуляровать студентов к ответу хотя бы на первые 60 вопросов, чтобы добиться устойчивого улучшения результатов и мотивации к дальнейшему обучению.

#С ростом кол-ва просмотренных объяснений устойчиво растет успеваемость студентов.

#Если студент посмотрит более 4х лекций, его успеваемость значительно возрастет. При этом просмотр более 11 лекций по какой-то причине не улучшает ситуацию.

#3. Добавим к исследованию данные из дополнительных файлов
#Исследуем данные "Вопросы"
#questions.csv

#question_id: внешний ключ вопроса - соответствует content_id
#bundle_id: код, по которому вопросы отдаются совместно
#correct_answer: правильный ответ. Можно использовать для проверки правильности user_answer
#part: номер раздела в тесте TOEIC
#tags: кодировка типа вопроса, можно использовать для кластеризации вопросов


questions = pd.read_csv('questions.csv', sep=',',
                        dtype = {'question_id': 'int16', 'bundle_id': 'int16', 'correct_answer': 'int8',
                                 'part': 'int8', 'tags': 'object'
                               })
questions.info()

#уникальные значения для колонок

unique_list_q = []
for col in questions.columns:
    item = (col, questions[col].nunique(), questions[col].dtype)
    unique_list_q.append(item)
unique_counts_q = pd.DataFrame(unique_list_q,
                               columns=['Column_Name', 'Num_Unique', 'Type']
                              ).sort_values(by='Num_Unique',  ignore_index=True)
display(unique_counts_q)

questions['question_id'].groupby(questions['bundle_id']).count().max()
questions['part'].value_counts(normalize=True)
questions['tags'].describe()

# Часть вопросов объединены по bundle_id в блоки до 5 вопросов, хотя большинство представлены по одиночке
# Задачи разделены по темам на 7 разделов, больше всего посвящено 5му разделу, 2, 3 и 4


q_tags = set()
for tag in questions['tags']:
    try:
        for t in tag.split():
                q_tags.add(int(t))
    except:
        q_tags.add(int(t))
len(q_tags)

# Можно провести доп исследование вопросов, используя кластеризацию по 188 доп.признакам 'tags'


#Таблица с ID из train
tmp_df = train.loc[(train.content_type_id == 0), ['content_id', 'answered_correctly']]

q_list = list(tmp_df['content_id'].unique())
len(q_list)

# Всего 13523 уникальных вопросов. Мы берем этот список из файла Train, чтобы иметь правильный порядок данных

q_quant = []
correct_quant = []
for q in q_list:
    tmp = tmp_df[tmp_df['content_id'] == q]['answered_correctly'].count()
    tmp1 = tmp_df[tmp_df['content_id'] == q]['answered_correctly'].sum()
    q_quant.append(tmp)
    correct_quant.append(tmp1)
    
    
q_ex = pd.DataFrame({'question_id': q_list,
                        'q_quant': q_quant,
                        'correct_quant': correct_quant
                       }
                      )
q_ex = q_ex.astype({'question_id': 'int16', 'q_quant': 'int32', 'correct_quant': 'int32'}
                   
                   
questions = questions.drop('correct_answer', axis=1)
questions = pd.merge(questions, q_ex, how='inner')
                   
                   
questions['correct_percent'] = questions['correct_quant'] / questions['q_quant']
questions.describe()
                   
                   
questions
                   
                   
#Дополнительные условия от которых может зависеть успеваемость
questions.groupby('part').mean()['correct_percent'].sort_values()

# По мере продвижения по разделам, видимо, сложность курса возрастает. 
# 5й раздел содержит максимальное число леций и заданий, однако средняя успеваемость студентов
# для него самая низкая.
                   
#ЛЕКЦИИ
                   
lectures = pd.read_csv('lectures.csv', sep=',',
                      dtype = {'lecture_id': 'int16', 'tag': 'int16', 
                               'part': 'int8', 'type_of': 'object'})
lectures.info()
                   
                   
#уникальные значения для колонок
                   
unique_list_lec = []
for col in lectures.columns:
    item = (col, lectures[col].nunique(), lectures[col].dtype)
    unique_list_lec.append(item)
unique_counts_lec = pd.DataFrame(unique_list_lec,
                                 columns=['Column_Name', 'Num_Unique', 'Type']
                                ).sort_values(by='Num_Unique',  ignore_index=True)
display(unique_counts_lec)

# Лекции разделены по темам на 7 разделов, больше всего лекций посвящено 5му разделу, затем 6, 2 и 1
# С учетом данных по вопросам, можно прийти к заключению, что 5 раздел - самый насыщенный по материалу,
# 6 и 1 - более теоретические, а 3 и 4 - более прикладные.
# Леции бывают 4 типов: вступление, целеполагание, концептуальное изложение материала и решение задач.
# Большинство лекций посвящены теории, немного меньше - решению задач. Доля остальных несущественна.
# Есть 151 доп.тип лекций, по которым можно провести кластеризацию
                   
    
                   
lectures['part'].value_counts(normalize=True)
lectures['type_of'].value_counts(normalize=True)
lectures['tag'].value_counts().head()
                   
                   
l_list = list(train[train['content_type_id'] == 1]['content_id'].unique())
len(l_list)

# Всего 415 уникальных лекций. Мы берем этот список из файла Train, чтобы иметь правильный порядок данных
                   
                   
l_quant = []
for l in l_list:
    tmp_l = train[(train['content_id'] == l)]['content_id'].count()
    l_quant.append(tmp_l)
                   
l_ex = pd.DataFrame({'lecture_id': l_list,
                        'l_quant': l_quant
                       }
                      )
l_ex = l_ex.astype({'lecture_id': 'int16', 'l_quant': 'int32'})
                   
                   
lectures = pd.merge(lectures, l_ex, how='inner')
display(lectures)
                   
                   
lectures.sort_values(by=['l_quant'], ascending=False).head()
        
lectures.groupby('part').sum()['l_quant'].sort_values(ascending=False)
                   
for part in range(1, 8):
    print(part, lectures.groupby('part').sum()['l_quant'].sort_values(ascending=False)[part]/lectures['part'].value_counts()[part])
    
# Наибольшее число просмотров имеет лекция из 7го раздела, 
# при этом самая высокая средневзвешенная популярность у лекций 2 раздела
                   
lectures.groupby('tag').sum()['l_quant'].sort_values(ascending=False).head()
                   
#новая таблица
                   
part_list = list(range(1, 8))
ques_quant_p = [questions[questions.part == p]['question_id'].count() for p in range(1,8)]
ans_quant_p = [questions[questions.part == p]['q_quant'].sum() for p in range(1,8)]
right_quant_p = [questions[questions.part == p]['correct_quant'].sum() for p in range(1,8)]
right_perc_p = [round(questions[questions.part == p]['correct_quant'].sum()/questions[questions.part == p]['q_quant'].sum(), 3) for p in range(1,8)]
lec_quant_p = [lectures[lectures.part == p]['lecture_id'].count() for p in range(1,8)]
lec_view_p = [lectures[lectures.part == p]['l_quant'].sum() for p in range(1,8)]
lec_pop_p = [lectures[lectures.part == p]['l_quant'].sum()/lectures[lectures.part == p]['lecture_id'].count() for p in range(1,8)]
norm_lec_pop_p = [round(pop / max(lec_pop_p), 3) for pop in lec_pop_p]
part_df = pd.DataFrame({'part': part_list,
                        'ques_quant_p': ques_quant_p,
                        'ans_quant_p': ans_quant_p,
                        'right_quant_p': right_quant_p,
                        'right_perc_p': right_perc_p,
                        'lec_quant_p': lec_quant_p,
                        'lec_view_p': lec_view_p,
                        'norm_lec_pop_p': norm_lec_pop_p
                       }
                      )
part_df = part_df.astype({'part': 'int8',
                        'ques_quant_p': 'int16',
                        'ans_quant_p': 'int64',
                        'right_quant_p': 'int64',
                        'right_perc_p': 'float32',
                        'lec_quant_p': 'int16',
                        'lec_view_p': 'int32',
                        'norm_lec_pop_p': 'float32'
                       })
                   
part_df.sort_values(by='right_perc_p', ascending=False)
    
#Основные выводы по разделу 3:
#Часть вопросов объединены по bundle_id в блоки до 5 вопросов, хотя большинство представлены по одиночке
#Лекции бывают 4 типов: вступление, целеполагание, концептуальное изложение материала и решение задач.
#Задачи и лекции разделены по темам на 7 разделов
#Большинство лекций посвящены теории, немного меньше - решению задач. Доля остальных несущественна.
#Наибольшее число просмотров имеет лекция из 7го раздела.
#Самая высокая средневзвешенная популярность у лекций 2 раздела.
#5 раздел - самый насыщенный по материалу, 6 и 1 - более теоретические, а 3 и 4 - более прикладные.
#По мере продвижения по разделам, видимо, сложность курса возрастает.
#5й раздел содержит максимальное число лекций и заданий, однако средняя успеваемость студентов для него самая низкая.
#Успеваемость по 4му и 6му разделу, вероятно, можно улучшить, добавив в них качественные лекции, а по 7му разделу - добавив практические задания.

SyntaxError: invalid character '№' (U+2116) (3316271523.py, line 138)